In [133]:
#imports
!pip install python-sat
from pysat.solvers import Minisat22 as sat
import numpy as np
from itertools import combinations as comb
from copy import deepcopy as dpc
import scipy.special as sp
from time import process_time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [134]:
# Encode the number into consecutive unique integers based on the value and its position
def encode_number(number, pos, size):
  return (number * size) + pos + 1 

# Find the number of digits which are correct as value and position
def find_correct_digit(encoded_guess, encoded_secret_sequence):
  num_correct_digit= 0
  for i in encoded_guess:
    if i in encoded_secret_sequence:
      num_correct_digit = num_correct_digit+1
  return num_correct_digit

# Encode the guess into consecutive unique integers based on the value and its position
def encode_guess(guess):
  return [(int(x)*len(guess) + i + 1) for i,x in enumerate(guess)]

# Update solver in order to have exactly one number in each position
def updated_solver_exactly_one_digit(guess, sat_solver):
    guess_length = len(guess)
    #Each position should have at least a number
    for r in range(guess_length):
      sat_solver.add_clause([encode_number(n,r,guess_length) for n in range(10)])
    #Each position can not have more than one value
    for r in range(guess_length):
      for n1 in range(10):
        for n2 in range(10): 
          if n1 != n2:
            sat_solver.add_clause([-encode_number(n1,r,guess_length),-encode_number(n2,r,guess_length)])
    return sat_solver

# Update solver by adding clauses for the correct digit/correct position combinations
def updated_solver_correct_clauses(guess, correct_digit, sat_solver):
    if correct_digit == 0:
      for x in guess:
        sat_solver.add_clause([int(-x)])
    else:
      cnf_least = list(comb(guess,len(guess) - int(correct_digit) + 1)) #AT LEAST K CORRECT DIGIT
      guess_negation = [ int(-x) for x in guess]
      cnf_most = list(comb(guess_negation, 1 + int(correct_digit))) #AT MOST K CORRECT DIGIT

      for cl in cnf_least:
        sat_solver.add_clause([cl1 for cl1 in list(cl)])
      for cm in cnf_most:
        sat_solver.add_clause([cm1 for cm1 in list(cm)])
      
    return sat_solver 

In [135]:
def NumberMind():
    t_start = process_time() 
    round = 1     
    num_correct_digit = 0
    not_found = True
    encoded_guess = []

    length_secret_sequence = np.random.randint(2,10)
    secret_sequence = np.random.randint(0,10,length_secret_sequence) 
    print("The Secret Sequence: ", secret_sequence)
    encoded_secret_sequence = encode_guess([int(x) for x in list(secret_sequence)]) # Encodes the secret code

    sat_solver = sat()    
    while not_found:
      new_guess = []
      print("\nROUND ", round, "")
      print("\n----PLAYER----")
      if round == 1:
        first_guess = np.random.randint(0,10,length_secret_sequence)   #the first guess is random
        new_guess = encode_guess(first_guess)
        if new_guess != encoded_secret_sequence:
         sat_solver = updated_solver_exactly_one_digit(new_guess,sat_solver)
      else:
        sat_solver = updated_solver_correct_clauses(encoded_guess, num_correct_digit, sat_solver)  #compute the correctnes (right digit/right position) clauses
        sat_solver.solve()
        for model in sat_solver.enum_models():
          new_guess = [x for x in model if x>0 ]
          break

      #Updating for new loop
      encoded_guess = new_guess
      print("guess",encoded_guess)

      print("\n----SECRET CODE OWNER----")
      # find the number of correct digit
      num_correct_digit= find_correct_digit(encoded_guess,encoded_secret_sequence)
      print("The number of correct digits: ", num_correct_digit)      #print the number of correct digits in right positions

#check is the number of correct digits in the right positions match the length of the solution: if so, the solution has been guessed correctly

      if num_correct_digit == length_secret_sequence:      
        not_found = False
        print("\nPLAYER is Master Mind by wining the game in", round, "rounds!")

        print("=======================================")
        t_stop = process_time()
        time_GD = t_stop - t_start
        print("CPU time:", time_GD)

      round+=1      #count the rounds


  

In [137]:
NumberMind()

The Secret Sequence:  [0 1 2 2 4 1 1 2]

ROUND  1 

----PLAYER----
guess [73, 50, 67, 28, 61, 46, 79, 32]

----SECRET CODE OWNER----
The number of correct digits:  0

ROUND  2 

----PLAYER----
guess [1, 2, 3, 4, 5, 6, 7, 8]

----SECRET CODE OWNER----
The number of correct digits:  1

ROUND  3 

----PLAYER----
guess [2, 11, 12, 13, 14, 15, 16, 65]

----SECRET CODE OWNER----
The number of correct digits:  2

ROUND  4 

----PLAYER----
guess [8, 11, 12, 21, 22, 23, 57, 74]

----SECRET CODE OWNER----
The number of correct digits:  0

ROUND  5 

----PLAYER----
guess [7, 16, 30, 65, 66, 75, 76, 77]

----SECRET CODE OWNER----
The number of correct digits:  0

ROUND  6 

----PLAYER----
guess [2, 15, 49, 59, 68, 69, 78, 80]

----SECRET CODE OWNER----
The number of correct digits:  1

ROUND  7 

----PLAYER----
guess [4, 13, 14, 41, 51, 58, 71, 80]

----SECRET CODE OWNER----
The number of correct digits:  1

ROUND  8 

----PLAYER----
guess [2, 14, 19, 31, 33, 53, 60, 72]

----SECRET CODE OWNER----